# Credit Score Classification
Bu projede Kredi Skoru Sınıflandırma modeli geliştiriliyor.<br>
Bu modelde kullanılan veriler <a href="https://thecleverprogrammer.com/2022/12/05/credit-score-classification-with-machine-learning/"
target="_blank">buradan</a> indirilmiştir.<br><br>
Bankaların ve kredi kartı şirketlerinin müşterilerini etiketlemek için kullandıkları üç kredi puanı vardır:
* Good
* 
Standar
* 
Por


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("train.csv")

### Keşfedici Veri Analizi (EDA)

In [3]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,5634,3392,1,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,26.822620,265.0,No,49.574949,21.46538,High_spent_Small_value_payments,312.494089,Good
1,5635,3392,2,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.944960,266.0,No,49.574949,21.46538,Low_spent_Large_value_payments,284.629162,Good
2,5636,3392,3,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,28.609352,267.0,No,49.574949,21.46538,Low_spent_Medium_value_payments,331.209863,Good
3,5637,3392,4,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.377862,268.0,No,49.574949,21.46538,Low_spent_Small_value_payments,223.451310,Good
4,5638,3392,5,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,24.797347,269.0,No,49.574949,21.46538,High_spent_Medium_value_payments,341.489231,Good


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  int64  
 1   Customer_ID               100000 non-null  int64  
 2   Month                     100000 non-null  int64  
 3   Name                      100000 non-null  object 
 4   Age                       100000 non-null  float64
 5   SSN                       100000 non-null  float64
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  float64
 8   Monthly_Inhand_Salary     100000 non-null  float64
 9   Num_Bank_Accounts         100000 non-null  float64
 10  Num_Credit_Card           100000 non-null  float64
 11  Interest_Rate             100000 non-null  float64
 12  Num_of_Loan               100000 non-null  float64
 13  Type_of_Loan              100000 non-null  ob

### Kategorik alanları sayıya dönüştürme
Yukarıda hiç bir alanda boş veri olmadığı görülmektedir. Bu nedenle sadece alanları sayıya çevirmek yeterli olacaktır.

In [10]:
df["Occupation"].value_counts()
# Bu alan dummy yapılabilir ya da eğitim verisine dahil edilmeyebilir

Occupation
Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6720
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: count, dtype: int64

In [12]:
df["Type_of_Loan"].value_counts()
# Bu alanı model eğitim zamanı listeden çıkaralım.

Type_of_Loan
No Data                                                                                                                               11408
Not Specified                                                                                                                          1408
Credit-Builder Loan                                                                                                                    1280
Personal Loan                                                                                                                          1272
Debt Consolidation Loan                                                                                                                1264
                                                                                                                                      ...  
Not Specified, Mortgage Loan, Auto Loan, and Payday Loan                                                                                  8
Payday 

In [13]:
df["Credit_Mix"].value_counts()

Credit_Mix
Standard    45848
Good        30384
Bad         23768
Name: count, dtype: int64

In [14]:
d={"Bad":0, "Standard":1, "Good":2}
df["Credit_Mix"]=df["Credit_Mix"].map(d)

In [15]:
df["Payment_of_Min_Amount"].value_counts()

Payment_of_Min_Amount
Yes    52326
No     35667
NM     12007
Name: count, dtype: int64

In [16]:
d={"No":0, "NM":1, "Yes":2}
df["Payment_of_Min_Amount"]=df["Payment_of_Min_Amount"].map(d)

In [17]:
df["Payment_Behaviour"].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      28616
High_spent_Medium_value_payments    19738
High_spent_Large_value_payments     14726
Low_spent_Medium_value_payments     14399
High_spent_Small_value_payments     11764
Low_spent_Large_value_payments      10757
Name: count, dtype: int64

In [18]:
d={
    "Low_spent_Small_value_payments":0,
    "Low_spent_Medium_value_payments":1,
    "Low_spent_Large_value_payments":2,
    "High_spent_Small_value_payments":3,
    "High_spent_Medium_value_payments":4,
    "High_spent_Large_value_payments":5
}
df["Payment_Behaviour"]=df["Payment_Behaviour"].map(d)

In [19]:
df["Credit_Score"].value_counts(dropna=False)

Credit_Score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

In [20]:
d={"Poor": 0, "Standard": 1, "Good": 2}
df["Credit_Score"]=df["Credit_Score"].map(d)

In [21]:
df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

## Modelleme

In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

g = GaussianNB()
b = BernoulliNB()
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
gb= GradientBoostingClassifier()
kn= KNeighborsClassifier()

def algo_test(x, y):
    modeller=[g, b, l, d, r, gb, kn]
    isimler=["GaussianNB", "BernoulliNB", "LogisticRegression", "DecisionTreeClassifier", 
             "RandomForestClassifier", "GradientBoostingClassifier", "KNeighborsClassifier"]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state = 42)
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
    mdl=[]

    print("Veriler hazır modeller deneniyor")
    for model in modeller:
        print(model, " modeli eğitiliyor!..")
        model=model.fit(x_train,y_train)
        tahmin=model.predict(np.array(x_test) if model==kn else x_test)
        mdl.append(model)
        accuracy.append(accuracy_score(y_test, tahmin))
        precision.append(precision_score(y_test, tahmin, average="micro"))
        recall.append(recall_score(y_test, tahmin, average="micro"))
        f1.append(f1_score(y_test, tahmin, average="micro"))
        print(confusion_matrix(y_test, tahmin))

    print("Eğitim tamamlandı.")
    
    metrics=pd.DataFrame(columns=["Accuracy", "Precision", "Recall", "F1", "Model"], index=isimler)
    metrics["Accuracy"] = accuracy
    metrics["Precision"] = precision  
    metrics["Recall"] = recall
    metrics["F1"] = f1
    metrics["Model"]=mdl

    metrics.sort_values("F1", ascending=False, inplace=True)

    print("En başarılı model: ", metrics.iloc[0].name)
    model=metrics.iloc[0,-1]
    tahmin=model.predict(np.array(x_test) if model==kn else x_test)
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, tahmin))
    print("classification Report:")
    print(classification_report(y_test, tahmin))
    print("Diğer Modeller:")
    
    return metrics.drop("Model", axis=1)

In [31]:
x=df[["Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Num_of_Loan", 
    "Delay_from_due_date", "Num_of_Delayed_Payment", "Credit_Mix", "Outstanding_Debt","Credit_History_Age", 
      "Payment_of_Min_Amount", "Payment_Behaviour", "Monthly_Balance"]]
y=df[["Credit_Score"]]

In [32]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Annual_Income           100000 non-null  float64
 1   Monthly_Inhand_Salary   100000 non-null  float64
 2   Num_Bank_Accounts       100000 non-null  float64
 3   Num_Credit_Card         100000 non-null  float64
 4   Interest_Rate           100000 non-null  float64
 5   Num_of_Loan             100000 non-null  float64
 6   Delay_from_due_date     100000 non-null  float64
 7   Num_of_Delayed_Payment  100000 non-null  float64
 8   Credit_Mix              100000 non-null  int64  
 9   Outstanding_Debt        100000 non-null  float64
 10  Credit_History_Age      100000 non-null  float64
 11  Payment_of_Min_Amount   100000 non-null  int64  
 12  Payment_Behaviour       100000 non-null  int64  
 13  Monthly_Balance         100000 non-null  float64
dtypes: float64(11), int64

In [25]:
x.shape, y.shape

((100000, 12), (100000, 1))

In [33]:
algo_test(x,y)

Veriler hazır modeller deneniyor
GaussianNB()  modeli eğitiliyor!..
[[6178 1189 1438]
 [3524 7871 4478]
 [ 127  974 4221]]
BernoulliNB()  modeli eğitiliyor!..
[[ 4359  3954   492]
 [ 2794 11686  1393]
 [  103  3784  1435]]
LogisticRegression()  modeli eğitiliyor!..
[[ 2873  5893    39]
 [ 2439 13138   296]
 [  105  4961   256]]
DecisionTreeClassifier()  modeli eğitiliyor!..
[[ 6353  2290   162]
 [ 2134 12064  1675]
 [  159  1653  3510]]
RandomForestClassifier()  modeli eğitiliyor!..
[[ 7304  1461    40]
 [ 1860 12897  1116]
 [   30  1290  4002]]
GradientBoostingClassifier()  modeli eğitiliyor!..
[[ 5962  2202   641]
 [ 2518 11361  1994]
 [  130  1465  3727]]
KNeighborsClassifier()  modeli eğitiliyor!..
[[ 7063  1669    73]
 [ 2214 12276  1383]
 [  200  1755  3367]]
Eğitim tamamlandı.
En başarılı model:  RandomForestClassifier
Confusion Matrix:
[[ 7304  1461    40]
 [ 1860 12897  1116]
 [   30  1290  4002]]
classification Report:
              precision    recall  f1-score   support

  

,Accuracy,Precision,Recall,F1
RandomForestClassifier,0.806767,0.806767,0.806767,0.806767
KNeighborsClassifier,0.756867,0.756867,0.756867,0.756867
DecisionTreeClassifier,0.730900,0.730900,0.730900,0.730900
GradientBoostingClassifier,0.701667,0.701667,0.701667,0.701667
GaussianNB,0.609000,0.609000,0.609000,0.609000
BernoulliNB,0.582667,0.582667,0.582667,0.582667
LogisticRegression,0.542233,0.542233,0.542233,0.542233


# Sonuç
Müşteri bilgileri kullanılarak Kredi Skoru Sınıfı tahmin eden model geliştirildi. Denenen sınıflandırma modelleri içerisinde
en yüksek başarı RandomForestClassifier modeli ile %81 oranında sağlandı.